# Strain mapping with all calibrations

The notebooks in this directory enable mapping strain in a material from 4D-STEM data with a full suite of calibrations.  The calibrations here include

- rotational offsets: determining and accounting for rotational offset of real and diffraction space due to rotation of the beam in the projector system.  Without this step, the coordinate system of the strain measurements are not known, and the resulting strain maps are not interpretable in real space.

- elliptical distortions: any minor tilt of the optic axis with respect to the detector distorts circles into ellipses in the resulting diffraction patterns; in a well-aligned tool, this causes elliptical distortions of 2-3% or greater.  Obtaining the most accurate strain maps requires measuring and correcting these distortions.

- Q pixel calibations: careful measurement of the detector pixel size is necessary to obtain lattice measurements from diffraction data on an absolute scale.  Crystalline peaks of known spacing from a calibration sample are important, as in real samples (especially containing strain) the absolute local lattice spacings are often uncertain.  Without this step, only relative strain (e.g. compression/tension relative to some locally measured region) can be measured.

Note that the analysis described here works for single crystalline samples.  For polycrystalline or amorphous samples, additional processing is necessary.  On-axis data is required, such that Bragg disks in two, non-colinear directions are captured.  

Selection of microscope parameters is crucial, and optimal parameters will vary from one system to the next.  Some considerations:

- camera length: optimally, as many Bragg disks as possible are present.  Thus, if there is sufficient scattering / camera dynamic range to capture disks at high angles, a large camera angle may be useful.  This consideration should be balanced against ensuring that the beam spans sufficiently many pixels - a beam that is only a few pixels wide will limit the [precision with which disk positions can be measured](https://arxiv.org/abs/1907.05504).

- convergence angle: disks that are too close can compromise disk detection; the convergence angle should be chosen such that the disks do not overlap, and optimally are separated by at least ~1 disk spacing.

___

## Data

The data required for this analysis [are](https://xkcd.com/1429/) as follows:

##### 4D-STEM data

The 4D-STEM scan of the sample to be strain mapped.

##### Vacuum scan

A 4D-STEM scan over vacuum, using *identical* microscope parameters to those used in obtaining the 4D-STEM scan.  This includes identical convergence angles, camera lengths, and binning.  This is used to generate a template of the vacuum probe, which is used to locate the Bragg disks.  While a single diffraction pattern over vacuum is in principle sufficient, a small 4D-STEM scan over vacuum - perhaps ~10x10 scan positions - is a very good idea, to obtain a higher SNR average probe.

##### Calibration sample scan

A 4D-STEM scan of a standard calibration sample.  This should be a sample with (1) precisely known lattice spacings, and (2) sufficiently many polycrystalline domains within the scanned region that Bragg reflections in all directions are represented.  This latter requirement is important for elliptical distortion measurements.  A small 4D-STEM scan - again, perhaps ~10x10 scan positions - is sufficient.

##### HAADF / defocused CBED pattern

Two images are included here: first is an image of the sample - a HAADF scan taken immediately before or after the 4D-STEM scan is often easiest, though any STEM image (bright field, LAADF, MAADF...) should suffice.  A virtual image from the 4D-STEM can also be used.  Second, a CBED pattern which has been defocused until a shadow image of the same region in the HAADF data is visible.  The rotation between these two images are then used to calibrate the rotational offset between the sample plane and the diffraction plane, which is essential to obtaining meaningful strain maps.

___

## Analysis

The steps to perform this analysis are summarized below.

### 1. Prepare data

Prepare the data for subsequent analysis.

##### A. Preprocess

Filehandling. Optionally: binning, cropping, background subtraction, format conversion.

- Inputs:
  - raw 4D-STEM scan
- Outputs:
  - processed 4D-STEM scan
  
  
##### B. Get vacuum probe

Obtain a vacuum probe template from the vacuum scan, and prepare it for template matching against the data.  If no vacuum scan is available but the experimental data contains a region of vacuum, this can be used instead.  If neither a vacuum scan nor vacuum region are available, generating a synthetic probe is often possible, though this approach is more difficult and may reduce accuracy.

- Inputs:
  - vacuum scan
- Outputs:
  - vacuum probe template

### 2. Calibration

Perform the various calibrations.

##### A. Get Bragg disks

Find the Bragg disks in the calibration sample scan.

- Inputs:
  - vacuum probe template
  - calibration sample scan
- Outputs:
  - Bragg peaks, uncorrected, for the calibration sample
  - Bragg vector map, uncorrected, for the calibration sample
  
  
##### B. Scan and elliptical distortion measurement

Measure and correct and overall shifts of the diffraction patterns from the scanning of the beam.  Measure the elliptical distortion from the calibration data.  The elliptical distortion parameters are saved for later application to the 4D-STEM data scan.  The elliptical distortions are then corrected in this dataset.

- Inputs:
  - Bragg peaks, uncorrected, for the calibration sample
  - Bragg vector map, uncorrected, for the calibration sample
- Outputs:
  - Bragg peaks, corrected, for the calibration sample
  - Bragg vector map, corrected, for the calibration sample
  - elliptical distortion parameters
  
  
##### C. Pixel calibration

Measure the pixel size on the detector after elliptical distortion correction, using the known calibration sample spacings.

- Inputs:
  - Bragg vector map, corrected, for the calibration sample
- Outputs:
  - Q pixel size
  
  
##### D. Rotational Calibration

Measure the rotational offset between the sample plane and the diffraction plane.  Additionally determine whether there is a coordinate system flip between the two planes, which can occur computationally during the data writing or reading.  This calibration can be performed only once and subsequently applied to additional datasets, *provided that* any scan coil rotations are accounted for.

- Inputs:
  - HAADF / other STEM image
  - defocused CBED containing a shadow image
- Outputs:
  - RQ rotational offset
  - RQ flip

### 3. Measure Bragg scattering

##### A. Get Bragg disks

Find the Bragg disks in the experimental 4D-STEM data.

- Inputs:
  - vacuum probe template
  - experimental sample scan
- Outputs:
  - Bragg peaks, uncorrected, for the experimental sample
  - Bragg vector map, uncorrected, for the experimental sample
  
  
##### B. Correct scan and elliptical distortions

Measure and correct and overall shifts of the diffraction patterns from the scanning of the beam.  Correct the elliptical distortions using the measurements performed on the calibration sample.

- Inputs:
  - Bragg peaks, uncorrected, for the experimental sample
  - Bragg vector map, uncorrected, for the experimental sample
- Outputs:
  - Bragg peaks, corrected, for the experimental sample
  - Bragg vector map, corrected, for the experimental sample


### 4. Measure lattice strain

##### A. Initial lattice vector guess

Get a rough guess at the lattice vectors.

- Inputs:
  - Bragg peaks, corrected, for the experimental sample
- Outputs:
  - lattice vectors, initial guess
  
  
##### B. Calculate strain

Calculate the in-plane elements of the [infinitesimal strain matrix](https://en.wikipedia.org/wiki/Infinitesimal_strain_theory).  The calculation is performed by first fitting the lattice vectors at each scan position to the detected Bragg peaks, weighting the importance of each Bragg peak by its intensity.  The variation of the lattice vectors relative to a reference lattice at each scan position is then used used to find the strain.  The reference lattice can be some selected region of the data itself, or optionally can be input separately, e.g. using another scan of some unstrained sample.  Optionally if the detector pixel size has been calibrated, absolute-scale lattice parameter maps can be generated as well.

- Inputs:
  - Bragg peaks, corrected, for the experimental sample
  - lattice vectors, initial guess
  - RQ offset, RQ flip
  - Q pixel size (optional)
  - lattice vectors, reference (optional)
- Outputs:
  - strain maps (relative scale)
  - lattice parameter maps (absolute scale) (optional)
  

##### C. Visualize

Plot the results.

- Inputs:
  - strain maps
- Outputs:
  - pretty pretty pictures
  
___

## Memory use and the Jupyter notebooks

The format of the workflow demoed here, in which many small Jupyter notebooks are used to perform a single larger task - strain mapping - has a number of advantages.  One is that it is makes it easy to begin processing some data, get to a stopping place, save your progress, and return later.  Another is that it's very modular.

This approach also has some disadvantages.  One important point is that because each notebook is running its own independent instance of python, it is possible to run into a couple of related file access and memory use problems.  Things to be careful about:

- for large 4D-STEM scan, loading the data into a single notebook may take up a significant portion of your machine's memory.  *Before loading a large datacube into a new notebook, make sure to shutdown any other notebooks with a loaded datacube, to avoid memory errors.*  Closing the tab for a notebook will *not* shutdown the associated python instance; make sure to actually shutdown the kernel.

- HDF5 files can only be opened from one place at a time.  This means that if you open an HDF5 (e.g. with `browser = py4DSTEM.file.io.FileBrowser('filename.h5')`), you need to close it again (e.g. with `browser.close()`) before you'll be able to read or write to that file.  Make sure to close your HDF5 files when you're done using them to avoid OSErrors.